<a href="https://colab.research.google.com/github/takahashi0329/DeepLerarning2022Late/blob/main/Selenium%E3%81%A7%E7%94%BB%E5%83%8F%E5%8F%8E%E9%9B%862022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#seleniumを使って自動で画像検索し、結果を収集する

##必要なアプリのインストール
###Seleniumのインストール(バージョンを指定する)

In [1]:
!pip install selenium==4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### chromium-chromedriverのインストール

In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


## seleniumを実行して操作する

In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
# エラー処理用
from selenium.common.exceptions import NoSuchShadowRootException
from selenium.common.exceptions import ElementClickInterceptedException

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## ログ用に時刻を取得

In [4]:
tm_start = time.time() #処理時間計測用
dt_now = datetime.datetime.now() #現在の日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M') #書式設定して
print(dt_date_str)

2022/12/05 02:56


##検索に関する設定値を定義する

In [5]:
QUERY = '犬　フリー' #検索したいキーワード
LIMIT_DL_NUM = 100 #ダウンロード数の上限
SAVE_DIR = 'img/dog_' #出力フォルダ(自動生成させる)
FILE_NAME = 'dog_' #ファイル名(後ろに連番と拡張子をつけていく)
TIMEOUT = 60 #検索のタイムアウト(秒)
ACCESS_WAIT = 1 #アクセスする間隔(秒)←超重要
RETRY_NUM = 3 #クリックやrequestのリトライ回数
DRIVER_PATH = '/usr/bin/chromedriver' #chromedriver.exeへのパス

## Chromeをヘッドレスモードで起動する
ヘッドレスモード = ブラウザを画面に開かないで実行するモード

In [6]:
from selenium.webdriver.chrome import options
# optionの指定を準備
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
# 起動する
driver = webdriver.Chrome(DRIVER_PATH, options=options)
driver.implicitly_wait(TIMEOUT) #タイムアウトを設定
# ログに処理時間を出力
tm_driver = time.time()
print('WebDriver起動完了', f'{tm_driver - tm_start:.1f}s')

<ipython-input-6-6a4538561041>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH, options=options)


WebDriver起動完了 4.2s


##Googleの画像検索へアクセス

In [7]:
# QUERY条件付きでURLを定義する
url = f'https://www.google.com/search?q={QUERY}&tbm=isch'
# 組み立てたURLでアクセスして検索
driver.get(url)
# 時刻のログを取得
tm_geturl = time.time()
print('Google画像検索ページ取得', f'{tm_geturl - tm_driver:.1f}s')


Google画像検索ページ取得 4.8s


##検索結果の要素を取得していく

In [8]:
# ページのサムネイル画像の要素群を取得
#(islmpクラスがしていされたimg=サムネイルをすべて拾う)
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
# サムネイルのリストから各サムネイルのalt属性を取得していく
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
# alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
count = len(tmb_alts) - tmb_alts.count('')
print(count)

<ipython-input-8-2b1592267c58>:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')


48


##下にスクロールさせてさらに検索結果を表示させてまたカウント

In [9]:
# 定義したLIMIT_DL_NUM(ダウンロード数の上限)になるまでスクロールを繰り返す
while count < LIMIT_DL_NUM:
  # ページの一番下へスクロールして新しいサムネイル画像を表示させる
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(1) #スリープを繰り返す間隔(秒)を入れる←重要
  #(islmpクラスがしていされたimg=サムネイルをすべて拾う)
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')
  # サムネイルのリストから各サムネイルのalt属性を取得していく
  tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
  # alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
  count = len(tmb_alts) - tmb_alts.count('')
  print(count)
# ループ終わり

<ipython-input-9-3d4d99cc23c3>:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')


100


## サムネイル画像をクリックすると右側に開かれる領域をオブジェクトに取得

In [10]:
imgframe_elem = driver.find_elements_by_id(('islsp'))

<ipython-input-10-533ebf20c90c>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  imgframe_elem = driver.find_elements_by_id(('islsp'))


##画像へURLアクセスして保存する準備をする

In [11]:
#画像の保存フォルダを作成
os.makedirs(SAVE_DIR, exist_ok=True)
#HTTPリクエスト用のヘッダーを作る
HTTP_HEADERS = {'User-Aget':driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)
# ダウンロードしていきたい対象のファイル拡張子
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.gif')

{'User-Aget': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/107.0.5304.87 Safari/537.36'}


## ダウンロード中に利用する関数を定義する
### 画像の拡張子を取得する関数

In [12]:
from selenium.common import exceptions
# 拡張子を取得する関数
def get_extension(url):
  url_lower = url.lower()
  for img_ext in IMG_EXTS:#拡張子のリストから１つずつ比較
    if img_ext in url_lower:
      extension = '.jpg' if img_ext == '.jpeg' else img_ext
      break
    # if終わり
    else:
      exceptions = ''
    # for終わり
  return extension
# 関数終わり

###urlで指定した画像へアクセスし、フォルダ内のファイルとして書き込む

In [14]:
def download_image(url, path, loop):
  result = False #戻り値の変数
  for i in range(loop):
    try:
      r = requests.get(url, headers=HTTP_HEADERS, stream=True, timeout=10)
      r.raise_for_status()
      with open(path,'wb') as f: # バイナリ書き込みモードで開く
        f.write(r.content)
    except requests.exceptions.SSLError:
      print('***** SSL エラー')
      break # リトライしない
    except requests.exceptions.RequestException as e:
      print(f'***** requests エラー({e}): {i + 1}/{RETRY_NUM}')
      time.sleep(1)
    else:
      result = True
      break # try成功
    # ループ終わり    
  #　関数終わり

In [15]:
### サムネイルを取得し、画像ダウンロードの準備ができたタイミング
tm_thumbnails = time.time()
print('サムネイル画像取得とDL準備完了', f'{tm_thumbnails - tm_geturl:.1f}s')

サムネイル画像取得とDL準備完了 2101.0s


##ダウンロード処理

In [ ]:
EXCLUSION_URL = 'https//lh3.googleusercontent.com/' #除外対象URL
count = 0
url_list = [ ]

# サムネイルをクリック、画像リンクを取得、画像をDL&保存を繰り返す
for tmlelem, tmb_alt in zip(tmb_elems, tmb_alts):
  
# ループ終わり